In [ ]:
import requests
import pprint
import pandas as pd
import imaplib
import email

## Gmail Client

In [ ]:
import talenttrove.app.email.gmail 
import importlib
importlib.reload(talenttrove.app.email.gmail)
from talenttrove.app.email.gmail import Gmail
from datetime import datetime,timedelta

In [ ]:
username ="agarwalpratham2001@gmail.com"

#generated app password
app_password= "lgjc xmxv ixyr nvxx"
client = Gmail(username, app_password)
client.authenticate()

In [ ]:
from_date = (datetime.now() - timedelta(days=1)).strftime("%d-%b-%Y")  # 7 days ago
to_date = (datetime.now()).strftime("%d-%b-%Y")  # Today
ids = client.get_email_by_date(from_date=from_date, to_date=to_date)
data = client.parse_emails(ids)

In [ ]:
data

In [ ]:
client.close()

In [ ]:
import http.cookies
cookie_str="""_fbp=fb.1.1694943938226.514835020; user.id=eyJfcmFpbHMiOnsibWVzc2FnZSI6Ik16ZzJNRFE9IiwiZXhwIjpudWxsLCJwdXIiOiJjb29raWUudXNlci5pZCJ9fQ%3D%3D--52ae7b5f23c3b774776a63e18d12439744528751; _hjSessionUser_1996783=eyJpZCI6Ijc0Nzk0Y2UwLWE4MTUtNWVhOC1iMDAxLTg1NmQyYTU2OWVjZCIsImNyZWF0ZWQiOjE2OTQ5NDQwMTE0MjksImV4aXN0aW5nIjp0cnVlfQ==; _gid=GA1.2.1688672101.1696942457; _lr_uf_-hoyxj9=97d909d9-820b-4748-8581-51a0150e803c; _lr_tabs_-hoyxj9%2Fnodeflair={%22sessionID%22:0%2C%22recordingID%22:%225-5dbb285d-e594-4c76-b3ef-89ae6c813be1%22%2C%22webViewID%22:null%2C%22lastActivity%22:1696942545714}; _lr_hb_-hoyxj9%2Fnodeflair={%22heartbeat%22:1696942545714}; cf_clearance=SRqUHV0_JaaYnxWUWGdwUY4agtH6lhcb4wbziRx_v20-1697001805-0-1-8d575e66.bf525c38.9e655532-0.2.1697001805; _hjSession_1996783=eyJpZCI6IjFmMDEyZTQ2LWQyMzItNGM5Ny04MjA5LTlkNmM4OGQ5Mjk5OCIsImNyZWF0ZWQiOjE2OTcwMDE4Nzg3NDYsImluU2FtcGxlIjpmYWxzZSwic2Vzc2lvbml6ZXJCZXRhRW5hYmxlZCI6ZmFsc2V9; _nodeflair_session=SgQmOwi2%2FsrWTlqvMKC6Rr%2F1C2mPiIZdQBm%2B5aWyyHMdth6kujmWtaNchOpglyULeo1nOufxew%2FdRULkNOs%2Bbwk%2BjPz3VQMFlI0gO5sXT0XfNvJE3ZpqzBHa7AF%2B%2BoGcx%2BQ5zOgaqtmd9kr5vAMDO%2FgUC2qOf3rOj9TxOEDwS21XzISVWahxgQFJEhfVmrug%2BGze3lGy1HEatiPfmGwF86v2KEnFdMZhLGZQXw8SuaEhYVih5%2BYeX01zEy3GG1imgM2t2ocEP4MZn2rLQy1F81dsvbacklBlM5FU2LS5SoS%2F6M2qmKT%2FgkbftLeLYlLN0ByWdj9wyugOotZmOUlJB%2FXQ96iiS6%2FDmgqioJKsN8Pin9ZM%2Bu8meoCwqsHSS3gHeiphGx4wHSbZJL%2BEoKJkObmymhOlc4r3kfrPaS2mfhNqVtcW51Bn40n52bYqpzrNkplvu28FbfKaeX04PWCIwpXVf6lOAL1YVCcZ%2BHZWpdUdPPEXGsVwDES4cDnEqYKPFfTuQDELFUTfbrYf9pODWcts3YrEUA%3D%3D--Ij%2FDf1fLhAY1aizO--kFllUCg2W%2FJOPpgiSX84jQ%3D%3D; _ga_0D09HRRGVH=GS1.1.1697001878.8.1.1697002556.60.0.0; _ga=GA1.2.334756147.1694943938; _gat_gtag_UA_116290073_1=1; _hjIncludedInSessionSample_1996783=0"""
cookie = http.cookies.SimpleCookie()
cookie.load(cookie_str)
cookie_dict = {k: v.value for k, v in cookie.items()}
print(cookie_dict)

In [ ]:
headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36",
    "sec-fetch-mode": "cors",
    # "If-None-Match": 'W/"1297c0623cd0e995397a9f6796e346c4"',
    "Sec-Ch-Ua-Mobile": "?0",
    "Sec-Ch-Ua-Platform": "macOS",
    "Sec-Fetch-Dest": "empty",

}
params={
    "query": None,
    "page": 2,
    "sort_by": "recent",
    "countries[]": "Singapore",
}
response = requests.get('https://nodeflair.com/api/v2/jobs',headers=headers,params=params)

In [ ]:
response.status_code

In [ ]:
response = requests.get('https://nodeflair.com/api/v2/jobs/172520',headers=headers)

In [ ]:
response = requests.get("https://api.simplify.jobs/v2/job-list/",headers=headers)

In [ ]:
response.json()

In [ ]:
tech_response = requests.get("https://api.simplify.jobs/v2/company/?page=1&size=100&value=&workflow_completed=true",headers=headers,cookies=cookie_dict)

In [ ]:
tech_response.json()

In [ ]:
from talenttrove.scrapers.utils import HEADERS
import urllib
# TECH_COMPANIES = "https://nodeflair.com/blog/top-tech-companies-in-singapore"
# response = urllib3.(TECH_COMPANIES,headers=HEADERS,cookies=cookie_dict)

In [ ]:
response = urllib.request.urlopen(TECH_COMPANIES)

## Shortlisting Companies

- https://www.timesbusinessdirectory.com/company-listings?page=9
- https://www.kaggle.com/datasets/peopledatalabssf/free-7-million-company-dataset
- https://beta.data.gov.sg/collections/2/view

In [ ]:
company_df = pd.read_csv("output/companies_sorted.csv")

In [ ]:
company_df=company_df[company_df['industry'] == 'information technology and services']
company_df = company_df[~company_df['size range'].isin(['1 - 10','11 - 50','51 - 200'])]
company_df.drop(columns=['Unnamed: 0','locality']).reset_index(drop=True).to_csv('talenttrove/data/companies/tech_companies.csv')

In [ ]:
import importlib
import talenttrove.scrapers.datagov
importlib.reload(talenttrove.scrapers.datagov)
from talenttrove.scrapers.datagov import DataGovCollection

In [ ]:
collection = DataGovCollection(2)

In [ ]:
sing_data = collection.collect()

In [ ]:
filter_data = collection.filter(sing_data)

In [ ]:
filter_data.shape

In [ ]:
filter_data.to_csv('talenttrove/data/companies/singapore_companies.csv',index=False)

## Glassdoor

In [ ]:
params=[
    {
        "operationName": "JobSearchResultsQuery",
        "variables": {
            "excludeJobListingIds": [],
            "filterParams": [
                {
                    "filterKey": "fromAge",
                    "values": "1"
                }
            ],
            "keyword": "",
            "locationId": 3235921,
            "locationType": "CITY",
            "numJobsToShow": 20,
            "originalPageUrl": "https://www.glassdoor.sg/Job/singapore-singapore-jobs-SRCH_IL.0,19_IC3235921.htm",
            "parameterUrlInput": "IL.0,19_IC3235921",
            "seoFriendlyUrlInput": "singapore-singapore-jobs",
            "seoUrl": True
        },
        "query": "query JobSearchResultsQuery($excludeJobListingIds: [Long!], $keyword: String, $locationId: Int, $locationType: LocationTypeEnum, $numJobsToShow: Int!, $pageCursor: String, $pageNumber: Int, $filterParams: [FilterParams], $originalPageUrl: String, $seoFriendlyUrlInput: String, $parameterUrlInput: String, $seoUrl: Boolean) {\n  jobListings(\n    contextHolder: {searchParams: {excludeJobListingIds: $excludeJobListingIds, keyword: $keyword, locationId: $locationId, locationType: $locationType, numPerPage: $numJobsToShow, pageCursor: $pageCursor, pageNumber: $pageNumber, filterParams: $filterParams, originalPageUrl: $originalPageUrl, seoFriendlyUrlInput: $seoFriendlyUrlInput, parameterUrlInput: $parameterUrlInput, seoUrl: $seoUrl, searchType: SR}}\n  ) {\n    filterOptions\n    indeedCtk\n    jobListings {\n      ...JobView\n      __typename\n    }\n    jobListingSeoLinks {\n      linkItems {\n        position\n        url\n        __typename\n      }\n      __typename\n    }\n    jobSearchTrackingKey\n    jobsPageSeoData {\n      pageMetaDescription\n      pageTitle\n      __typename\n    }\n    paginationCursors {\n      cursor\n      pageNumber\n      __typename\n    }\n    indexablePageForSeo\n    searchResultsMetadata {\n      searchCriteria {\n        implicitLocation {\n          id\n          localizedDisplayName\n          type\n          __typename\n        }\n        keyword\n        location {\n          id\n          shortName\n          localizedShortName\n          localizedDisplayName\n          type\n          __typename\n        }\n        __typename\n      }\n      footerVO {\n        countryMenu {\n          childNavigationLinks {\n            id\n            link\n            textKey\n            __typename\n          }\n          __typename\n        }\n        __typename\n      }\n      helpCenterDomain\n      helpCenterLocale\n      jobAlert {\n        jobAlertExists\n        __typename\n      }\n      jobSerpFaq {\n        questions {\n          answer\n          question\n          __typename\n        }\n        __typename\n      }\n      jobSerpJobOutlook {\n        occupation\n        paragraph\n        __typename\n      }\n      showMachineReadableJobs\n      __typename\n    }\n    serpSeoLinksVO {\n      relatedJobTitlesResults\n      searchedJobTitle\n      searchedKeyword\n      searchedLocationIdAsString\n      searchedLocationSeoName\n      searchedLocationType\n      topCityIdsToNameResults {\n        key\n        value\n        __typename\n      }\n      topEmployerIdsToNameResults {\n        key\n        value\n        __typename\n      }\n      topEmployerNameResults\n      topOccupationResults\n      __typename\n    }\n    totalJobsCount\n    __typename\n  }\n}\n\nfragment JobView on JobListingSearchResult {\n  jobview {\n    header {\n      adOrderId\n      advertiserType\n      adOrderSponsorshipLevel\n      ageInDays\n      divisionEmployerName\n      easyApply\n      employer {\n        id\n        name\n        shortName\n        __typename\n      }\n      employerNameFromSearch\n      goc\n      gocConfidence\n      gocId\n      jobCountryId\n      jobLink\n      jobResultTrackingKey\n      jobTitleText\n      locationName\n      needsCommission\n      payCurrency\n      payPeriod\n      payPeriodAdjustedPay {\n        p10\n        p50\n        p90\n        __typename\n      }\n      rating\n      salarySource\n      savedJobId\n      sponsored\n      __typename\n    }\n    job {\n      importConfigId\n      jobTitleId\n      jobTitleText\n      listingId\n      __typename\n    }\n    jobListingAdminDetails {\n      cpcVal\n      importConfigId\n      jobListingId\n      jobSourceId\n      userEligibleForAdminJobDetails\n      __typename\n    }\n    overview {\n      shortName\n      squareLogoUrl\n      __typename\n    }\n    __typename\n  }\n  __typename\n}\n"
    }
]
response = requests.post("https://www.glassdoor.sg/graph",headers=headers,json=params)

In [ ]:
response.json()[0]['data']['jobListings']

In [ ]:
import http.cookies
cookie_str="""__Host-next-auth.csrf-token=198a9ac7d0efdde2761843e0ea10914e75be964787da340ea260f59e878143fd%7Cba8f77fca901b492e38df515db01fb6a544373d922c64a4c602ebb9dc214a723; _ga=GA1.1.81457931.1696941213; __Secure-next-auth.callback-url=https%3A%2F%2Fwww.carbosjobs.com%2F; _ga_JVWQGY8Q4P=GS1.1.1696959406.2.1.1696959543.0.0.0; __Secure-next-auth.session-token=eyJhbGciOiJkaXIiLCJlbmMiOiJBMjU2R0NNIn0..O0NVbTcl0mPJLdu4.Mhtrnhe1gjYLBYzQOEw_CSTUR2qlOTY4ZNupz6V9uxRAQ_GOYYF8CG1-WVNe8vgDTQdKTb9woqFATtlJZ4_XspQHRQJuSjlUZNAu09mlIF9hD1I788ghDLomOqgTyaxbVp5POSDltT-8wPeqe7Y4773wOO4OAoXR5-LBUmUWy2L1g1eG8gJAbkDAp_D0rBBaLzoQk1iNZKMApNWtovxtF6jyw6XNoNOIaokIuPQJAbamD-oB.fI5ORt-lR8ytWMhbi8mFvw; ph_phc_c3EWoh5aCGLJ7aaRb2TkaJOCIGhGJ1AhLMMLR9GJyT8_posthog=%7B%22distinct_id%22%3A%22clnkbhoce0000jw08m32ejv45%22%2C%22%24device_id%22%3A%22018b1994-004d-7b0a-a554-92d0f8c1f935%22%2C%22%24user_state%22%3A%22identified%22%2C%22%24sesid%22%3A%5B1696959551540%2C%22018b1aa9-92eb-7578-aef2-47c19040fd55%22%2C1696959402729%5D%2C%22%24session_recording_enabled_server_side%22%3Atrue%2C%22%24console_log_recording_enabled_server_side%22%3Atrue%2C%22%24session_recording_recorder_version_server_side%22%3A%22v2%22%2C%22%24autocapture_disabled_server_side%22%3Afalse%2C%22%24active_feature_flags%22%3A%5B%5D%2C%22%24enabled_feature_flags%22%3A%7B%7D%2C%22%24feature_flag_payloads%22%3A%7B%7D%2C%22%24user_id%22%3A%22clnkbhoce0000jw08m32ejv45%22%2C%22%24stored_person_properties%22%3A%7B%22email%22%3A%22agarwalpratham2001%40gmail.com%22%2C%22name%22%3A%22Pratham%20Agarwala%22%7D%7D"""
cookie = http.cookies.SimpleCookie()
cookie.load(cookie_str)
cookie_dict = {k: v.value for k, v in cookie.items()}
print(cookie_dict)

In [ ]:
response = requests.get("https://www.carbosjobs.com/api/job-postings/search?FTSQuery=%28%27new+graduate%27+%7C+%27new+grad%27+%7C+%27entry%27+%7C+%27junior%27+%7C+%27campus%27+%7C+%27early+career%27+%7C+%27university%27+%7C+%27college+grad%27+%7C+%27entry-level%27%29+%26+%28%27trading%27+%7C+%27quantitative%27+%7C+%27market%27+%7C+%27quant%27+%7C+%27trader%27%29+%26+%28%21%272023%27+%26+%21%27fall%27+%26+%21%27winter%27+%26+%21%27m%2Ff%2Fd%27+%26+%21%27senior%27+%26+%21%27intern%27+%26+%21%27internship%27+%26+%21%27internships%27+%26+%21%27data+entry%27%29&filterName=Quant+%26+Trading%2C+New+Grad+%28USA%29",headers=headers,cookies=cookie_dict)

In [ ]:
import json
len(json.loads(response.json()))

In [ ]:
https://www.google.com/search?q=google+jobs&oq=google+jobs&sourceid=chrome&ie=UTF-8&ibp=htl;jobs&sa=X&ved=2ahUKEwifttedluaBAxWpxDgGHdNwCRQQutcGKAF6BAgOEAY&sxsrf=AM9HkKlkNKxI2yk2kPylgS6nYtrhsl31Lg:1696758567924#htivxrt=jobs&fpstate=tldetail&htichips=date_posted:3days&htischips=date_posted;3days&htidocid=UbM2CbLbvu4OMmqWAAAAAA%3D%3D

In [ ]:
https://www.linkedin.com/jobs/search/?f_C=1009%2C155134%2C9263339%2C3634137&geoId=92000000&origin=COMPANY_PAGE_JOBS_CLUSTER_EXPANSION&originToLandingJobPostings=3703127521%2C3693596011%2C3693593672%2C3727737691%2C3705389269%2C3705383975%2C3715841313%2C3705385738%2C3705649976&lipi=urn%3Ali%3Apage%3Acompanies_company_jobs_jobs%3Bb802d511-1d27-453d-a8d8-51463c7c7316

## Carbos

In [ ]:
import importlib
import talenttrove.scrapers.carbos
importlib.reload(talenttrove.scrapers.carbos)
from talenttrove.scrapers.carbos import Carbos,ALL_ENGINEERING_GRAD,ALL_ENGINEERING_INTERNSHIPS,ALL_QUANT_TRADING_INTERNSHIPS,ALL_QUANT_TRADING_GRAD

In [ ]:
cookie_str="""__Host-next-auth.csrf-token=198a9ac7d0efdde2761843e0ea10914e75be964787da340ea260f59e878143fd%7Cba8f77fca901b492e38df515db01fb6a544373d922c64a4c602ebb9dc214a723; _ga=GA1.1.81457931.1696941213; __Secure-next-auth.callback-url=https%3A%2F%2Fwww.carbosjobs.com%2F; _ga_JVWQGY8Q4P=GS1.1.1696959406.2.1.1696959543.0.0.0; __Secure-next-auth.session-token=eyJhbGciOiJkaXIiLCJlbmMiOiJBMjU2R0NNIn0..O0NVbTcl0mPJLdu4.Mhtrnhe1gjYLBYzQOEw_CSTUR2qlOTY4ZNupz6V9uxRAQ_GOYYF8CG1-WVNe8vgDTQdKTb9woqFATtlJZ4_XspQHRQJuSjlUZNAu09mlIF9hD1I788ghDLomOqgTyaxbVp5POSDltT-8wPeqe7Y4773wOO4OAoXR5-LBUmUWy2L1g1eG8gJAbkDAp_D0rBBaLzoQk1iNZKMApNWtovxtF6jyw6XNoNOIaokIuPQJAbamD-oB.fI5ORt-lR8ytWMhbi8mFvw; ph_phc_c3EWoh5aCGLJ7aaRb2TkaJOCIGhGJ1AhLMMLR9GJyT8_posthog=%7B%22distinct_id%22%3A%22clnkbhoce0000jw08m32ejv45%22%2C%22%24device_id%22%3A%22018b1994-004d-7b0a-a554-92d0f8c1f935%22%2C%22%24user_state%22%3A%22identified%22%2C%22%24sesid%22%3A%5B1696959551540%2C%22018b1aa9-92eb-7578-aef2-47c19040fd55%22%2C1696959402729%5D%2C%22%24session_recording_enabled_server_side%22%3Atrue%2C%22%24console_log_recording_enabled_server_side%22%3Atrue%2C%22%24session_recording_recorder_version_server_side%22%3A%22v2%22%2C%22%24autocapture_disabled_server_side%22%3Afalse%2C%22%24active_feature_flags%22%3A%5B%5D%2C%22%24enabled_feature_flags%22%3A%7B%7D%2C%22%24feature_flag_payloads%22%3A%7B%7D%2C%22%24user_id%22%3A%22clnkbhoce0000jw08m32ejv45%22%2C%22%24stored_person_properties%22%3A%7B%22email%22%3A%22agarwalpratham2001%40gmail.com%22%2C%22name%22%3A%22Pratham%20Agarwala%22%7D%7D"""
client = Carbos(cookie_str)

In [ ]:
jobs_1 = client.collect(ALL_ENGINEERING_GRAD)
jobs_1 = pd.DataFrame(jobs_1)
jobs_2 = client.collect(ALL_ENGINEERING_INTERNSHIPS)
jobs_2 = pd.DataFrame(jobs_2)
jobs_3 = client.collect(ALL_QUANT_TRADING_INTERNSHIPS)
jobs_3 = pd.DataFrame(jobs_3)
jobs_4 = client.collect(ALL_QUANT_TRADING_GRAD)
jobs_4 = pd.DataFrame(jobs_4)

In [ ]:
pd.concat([jobs_1,jobs_2,jobs_3,jobs_4],axis=0).to_csv("talenttrove/data/jobs/carbos.csv")

## My CarrersFuture

In [ ]:
import importlib
import talenttrove.scrapers.mycareersfuture
importlib.reload(talenttrove.scrapers.mycareersfuture)
from talenttrove.scrapers.mycareersfuture import MyCareersFuture

In [ ]:
client = MyCareersFuture()

In [ ]:
data = client.collect(sleep_time=2)

In [ ]:
pd.DataFrame(data).to_json("talenttrove/data/jobs/mycareersfuture.json",orient="records")

## Simplify Company and Job Scraper

In [ ]:
import importlib
import talenttrove.scrapers.simplifyJobs
importlib.reload(talenttrove.scrapers.simplifyJobs)
from talenttrove.scrapers.simplifyJobs import SimplifyJobs

In [ ]:
from talenttrove.scrapers.utils import HEADERS

In [ ]:
response=requests.get(url=f"https://api.simplify.jobs/v2/company?size=100&page=1",headers=HEADERS)

In [ ]:
response.text

In [ ]:
client= SimplifyJobs()
jobs = client.collect()

## Techjob Repo Scrapper
- https://techjobrepo.com/

In [ ]:
import importlib
import talenttrove.scrapers.techjobrepo
importlib.reload(talenttrove.scrapers.techjobrepo)
from talenttrove.scrapers.techjobrepo import TechJobRepo

In [ ]:
client= TechJobRepo()
jobs = client.collect()

In [ ]:
import pandas as pd

In [ ]:
pd.json_normalize(jobs).to_csv("talenttrove/data/techjobrepo.csv",index=False)